# The Battle of Neighborhoods - Analysis

### 1. Installing Python Libraries

In [488]:
!pip install geocoder
!pip install folium

In [489]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 2. Data Preparation and Cleaning

In [490]:
# Get data of postal code in Canada
file = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_file = requests.get(file)

In [491]:
wiki_data = pd.read_html(wiki_file.text)
wiki_data

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 5           M6A        North York   
 6           M7A  Downtown Toronto   
 7           M8A      Not assigned   
 8           M9A         Etobicoke   
 9           M1B       Scarborough   
 10          M2B      Not assigned   
 11          M3B        North York   
 12          M4B         East York   
 13          M5B  Downtown Toronto   
 14          M6B        North York   
 15          M7B      Not assigned   
 16          M8B      Not assigned   
 17          M9B         Etobicoke   
 18          M1C       Scarborough   
 19          M2C      Not assigned   
 20          M3C        North York   
 21          M4C         East York   
 22          M5C  Downtown Toronto   
 23          M6C              York   
 24          M7C      Not assigned   
 25         

In [492]:
# Get the data only on Postal code and Borough wih Neighbourhood
wiki_data = wiki_data[0]
wiki_data

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [493]:
# Create the dataframe based by Borough without 'Not assigned values'
df = wiki_data[wiki_data["Borough"] != "Not assigned"]
df.head

<bound method NDFrame.head of     Postal Code           Borough  \
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
8           M9A         Etobicoke   
9           M1B       Scarborough   
11          M3B        North York   
12          M4B         East York   
13          M5B  Downtown Toronto   
14          M6B        North York   
17          M9B         Etobicoke   
18          M1C       Scarborough   
20          M3C        North York   
21          M4C         East York   
22          M5C  Downtown Toronto   
23          M6C              York   
26          M9C         Etobicoke   
27          M1E       Scarborough   
30          M4E      East Toronto   
31          M5E  Downtown Toronto   
32          M6E              York   
36          M1G       Scarborough   
39          M4G         East York   
40          M5G  Downtown Toronto   
41      

In [494]:
#Grouping the dataframe based on Borough
df = df.groupby(['Borough','Postal Code'])['Neighbourhood'].apply(', '.join).reset_index()
df.head()

,Borough,Postal Code,Neighbourhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,"North Toronto West, Lawrence Park"
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park, Summerhill East"


In [495]:
# Checking df datatypes
df.dtypes

Borough          object
Postal Code      object
Neighbourhood    object
dtype: object

In [496]:
df.describe()

,Borough,Postal Code,Neighbourhood
count,103,103,103
unique,10,103,99
top,North York,M1V,Downsview
freq,24,1,4


In [497]:
#Counting Borough with the most postal codes

counts = df["Borough"].value_counts()
counts

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

From the analysis above founded that `North York` is the Borough with the most postal codes on Canada

So we're gonna sample `North York` as destination

In [498]:
def get_latilong(Borough):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(Borough))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('North York')

[43.768260000000055, -79.41262999999998]

In [499]:
import geocoder

In [500]:
import pandas as pd

geo = pd.read_csv(r"D:\Dhitya\Dhitya\Tutorial\Coursera\Data Science by IBM\09. Applied Data Science Capstone\Geospatial_Coordinates.csv")
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [501]:
geo_df = df.join(geo.set_index('Postal Code'), on='Postal Code', how='inner')
geo_df.head()

,Borough,Postal Code,Neighbourhood,Latitude,Longitude
0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790
1,Central Toronto,M4P,Davisville North,43.712751,-79.390197
2,Central Toronto,M4R,"North Toronto West, Lawrence Park",43.715383,-79.405678
3,Central Toronto,M4S,Davisville,43.704324,-79.388790
4,Central Toronto,M4T,"Moore Park, Summerhill East",43.689574,-79.383160


In [502]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [503]:
from geopy.geocoders import Nominatim

address = 'North York, Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


print(latitude, longitude)

43.7543263 -79.44911696639593


In [504]:
import folium
map_nyork = folium.Map(location=[latitude, longitude], zoom_start=13)


for latitude, longitude, borough, neighbourhood in zip(geo_df['Latitude'], geo_df['Longitude'], geo_df['Borough'], geo_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=False
        ).add_to(map_nyork)  
    
map_nyork

Connection to Foursquare:

In [505]:
CLIENT_ID = 'DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY'
CLIENT_SECRET = '1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+CLIENT_SECRET)

Your credentails:
CLIENT_ID: DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY
CLIENT_SECRET: 1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B


In [506]:
radius = 700 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
   longitude, 
    radius, 
   LIMIT)
results = requests.get(url).json()

Get nearby venues at `North York`

In [507]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', 'venue.location.labeledLatLngs',
       'venue.location.distance', 'venue.location.cc', 'venue.location.city',
       'venue.location.state', 'venue.location.country',
       'venue.location.formattedAddress', 'venue.categories',
       'venue.photos.count', 'venue.photos.groups',
       'venue.location.crossStreet', 'venue.location.postalCode'],
      dtype='object')

In [508]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [509]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,P&M Restaurant,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",43.701108,-79.518982
1,Shoppers Drug Mart,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",43.701264,-79.519705
2,Popeyes Louisiana Kitchen,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",43.701045,-79.518921
3,Grattan Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.706222,-79.521705
4,Greenland Farm Supermarket,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",43.701635,-79.519201


Grouping nearby venues to some categories

In [510]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,P&M Restaurant,Diner,43.701108,-79.518982
1,Shoppers Drug Mart,Pharmacy,43.701264,-79.519705
2,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.701045,-79.518921
3,Grattan Park,Park,43.706222,-79.521705
4,Greenland Farm Supermarket,Grocery Store,43.701635,-79.519201
5,Weston Ford,Gift Shop,43.701671,-79.522106


In [511]:
# Top 10 Categories
a=pd.Series(nearby_venues.categories)
a.value_counts()[:10]

Grocery Store          1
Diner                  1
Fried Chicken Joint    1
Gift Shop              1
Park                   1
Pharmacy               1
Name: categories, dtype: int64

In [541]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # making GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [513]:
Nyork_venues = getNearbyVenues(names=geo_df['Neighbourhood'],
                                   latitudes=geo_df['Latitude'],
                                   longitudes=geo_df['Longitude'])

Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government
The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Business reply mail Processing Centre, South Central Letter 

In [542]:
print('There are {} Uniques Categories.'.format(len(Nyork_venues['Venue Category'].unique())))
Nyork_venues.groupby('Neighbourhood').count()

There are 320 Uniques Categories.


,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,12,12,12,12,12,12
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,8,8,8,8,8,8
"Bedford Park, Lawrence Manor East",29,29,29,29,29,29
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",8,8,8,8,8,8
"Brockton, Parkdale Village, Exhibition Place",86,86,86,86,86,86
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",44,44,44,44,44,44


### One Hot Encoding

In [564]:
# one hot encoding
Nyork_onehot = pd.get_dummies(Nyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Nyork_onehot['Neighbourhood'] = Nyork_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Nyork_onehot.columns[-1]] + list(Nyork_onehot.columns[:-1])
Nyork_onehot = Nyork_onehot[fixed_columns]
Nyork_grouped = Nyork_onehot.groupby('Neighbourhood').mean().reset_index()
Nyork_onehot.head(3)

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery 

In [566]:
#Top 3 Nearby Venues in Nearby Locations
num_top_venues = 3
for hood in Nyork_grouped['Neighbourhood']:
    print("---- "+hood+" ----")
    temp =Nyork_grouped[Nyork_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Agincourt ----
                 venue  freq
0  Shanghai Restaurant  0.08
1       Clothing Store  0.08
2       Breakfast Spot  0.08


---- Alderwood, Long Branch ----
               venue  freq
0  Convenience Store   0.2
1        Pizza Place   0.2
2                Pub   0.1


---- Bathurst Manor, Wilson Heights, Downsview North ----
         venue  freq
0         Bank  0.09
1  Coffee Shop  0.09
2  Pizza Place  0.04


---- Bayview Village ----
           venue  freq
0           Bank  0.25
1           Café  0.12
2  Grocery Store  0.12


---- Bedford Park, Lawrence Manor East ----
                venue  freq
0  Italian Restaurant  0.10
1         Coffee Shop  0.10
2      Sandwich Place  0.07


---- Berczy Park ----
         venue  freq
0  Coffee Shop  0.08
1        Hotel  0.06
2         Café  0.05


---- Birch Cliff, Cliffside West ----
             venue  freq
0             Park  0.25
1  Thai Restaurant  0.12
2     Skating Rink  0.12


---- Brockton, Parkdale Village, Exhibition Place

In [567]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [575]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Nyork_grouped['Neighbourhood']

for ind in np.arange(Nyork_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Nyork_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Pool Hall,Coffee Shop,Motorcycle Shop,Skating Rink,Lounge,Latin American Restaurant,Shanghai Restaurant,Breakfast Spot,Sandwich Place,Clothing Store
1,"Alderwood, Long Branch",Pizza Place,Convenience Store,Pub,Sandwich Place,Gym,Coffee Shop,Pharmacy,Gas Station,Eastern European Restaurant,Dog Run
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Sandwich Place,Deli / Bodega,Mobile Phone Shop,Supermarket,Sushi Restaurant,Shopping Mall,Fried Chicken Joint,Frozen Yogurt Shop
3,Bayview Village,Bank,Intersection,Chinese Restaurant,Café,Grocery Store,Skating Rink,Japanese Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Cosmetics Shop,Indian Restaurant,Butcher,Café,Pizza Place,Locksmith,Liquor Store


### K-Means Clustering Approach

In [576]:
Nyork_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery 

In [577]:
# Using K-Means to cluster neighborhood into 3 clusters
Nyork_grouped_clustering = Nyork_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=3, random_state=0).fit(Nyork_grouped_clustering)
kmeans.labels_


array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 2])

In [582]:
#insert cluster labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Nyork_merged =geo_df.loc[geo_df['Borough']=='North York']

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Nyork_merged = Nyork_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Nyork_merged

,Borough,Postal Code,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,North York,M2H,Hillcrest Village,43.803762,-79.363452,0,Pharmacy,Restaurant,Chinese Restaurant,Sandwich Place,Bakery,Farmers Market,Eastern European Restaurant,Doner Restaurant,Donut Shop,Filipino Restaurant
52,North York,M2J,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Juice Bar,Baseball Field,Shoe Store,Women's Store,Bank,Bakery
53,North York,M2K,Bayview Village,43.786947,-79.385975,1,Bank,Intersection,Chinese Restaurant,Café,Grocery Store,Skating Rink,Japanese Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant
54,North York,M2L,"York Mills, Silver Hills",43.757490,-79.374714,2,Martial Arts School,Yoga Studio,Event Space,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant
55,North York,M2M,"Willowdale, Newtonbrook",43.789053,-79.408493,1,Park,Trail,Coffee Shop,Yoga Studio,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
56,North York,M2N,"Willowdale, Willowdale East",43.770120,-79.408493,1,Coffee Shop,Ramen Restaurant,Pizza Place,Korean Restaurant,Japanese Restaurant,Sandwich Place,Café,Middle Eastern Restaurant,Fast Food Restaurant,Sushi Restaurant
57,North York,M2P,York Mills West,43.752758,-79.400049,1,Park,Intersection,Convenience Store,Bowling Alley,Pet Store,Gym,Yoga Studio,Drugstore,Dumpling Restaurant,Eastern European Restaurant
58,North York,M2R,"Willowdale, Willowdale West",43.782736,-79.442259,0,Butcher,Park,Pizza Place,Coffee Shop,Grocery Store,Pharmacy,Yoga Studio,Escape Room,Donut Shop,Drugstore
59,North York,M3A,Parkwoods,43.753259,-79.329656,1,Park,Food & Drink Shop,Burger Joint,Pet Store,Yoga Studio,Ethiopian Restaurant,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
60,North York,M3B,Don Mills,43.745906,-79.352188,1,Japanese Restaurant,Gym,Coffee Shop,Beer Store,Sporting Goods Shop,Nightclub,Italian Restaurant,Discount Store,Clothing Store,Chinese Restaurant


In [580]:
# Dropping columns
#Nyork_merged = Nyork_merged.drop(['1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue','4th Most Common Venue','5th Most Common Venue','6th Most Common Venue','7th Most Common Venue','8th Most Common Venue','9th Most Common Venue','10th Most Common Venue'], axis=1)


### Clustering Neighbourhood

In [581]:
kclusters = 10

# create map
map_clusters = folium.Map(location=[latitude_x, longitude_y], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map

markers_colors = []
for lat, lon, nei , cluster in zip(Nyork_merged['Latitude'], 
                                   Nyork_merged['Longitude'], 
                                   Nyork_merged['Neighbourhood'], 
                                   Nyork_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#4856fb', '#10a2f0', '#2adddd', '#62fbc4', '#9cfba4', '#d4dd80', '#ffa256', '#ff562c', '#ff0000']


In [583]:

df1=Nyork_merged.loc[Nyork_merged['Cluster Labels'] == 0,Nyork_merged.columns[[2] + list(range(5, Nyork_merged.shape[1]))]]
df2=Nyork_merged.loc[Nyork_merged['Cluster Labels'] == 1,Nyork_merged.columns[[2] + list(range(5, Nyork_merged.shape[1]))]]
df3=Nyork_merged.loc[Nyork_merged['Cluster Labels'] == 2,Nyork_merged.columns[[2] + list(range(5, Nyork_merged.shape[1]))]]

### School rating by Clusters

In [603]:
clusters=pd.DataFrame({"Cluster1":df1["Neighbourhood"],
                      "Cluster2":df2["Neighbourhood"],
                      "Cluster4":df3["Neighbourhood"]})
clusters = clusters.replace(np.nan, '', regex=True)

In [600]:
new_Nyork=Nyork_merged.set_index("Neighbourhood",drop=True)
#Source:https://www.greatschools.org
Nyork_school_ratings=pd.DataFrame({"Neighbourhood":df["Neighbourhood"],"Latitude":geo_df["Latitude"],"Longitude":geo_df["Longitude"],
                                      "Top School Rating":[7,9,5,8,10,10,7,10,1,2,1,2,7,2,3,2,6,
                                                           5,4,8,9,9,6,6,4,5,4,6,8,10,8,9,6,2,
                                                           10,2,5,8,9,6,6,10,8,9,1,2,3,4,5,6,9,
                                                           8,5,9,6,9,6,4,8,10,2,5,6,3,9,8,7,
                                                           7,8,5,8,9,1,5,4,7,2,3,6,6,9,4,8,7,
                                                           4,8,9,2,6,4,7,5,10,4,6,8,9,7,5,6,5,8,7
                                                           ]})

Nyork_school_ratings.head()

,Neighbourhood,Latitude,Longitude,Top School Rating
0,Lawrence Park,43.728020,-79.388790,7
1,Davisville North,43.712751,-79.390197,9
2,"North Toronto West, Lawrence Park",43.715383,-79.405678,5
3,Davisville,43.704324,-79.388790,8
4,"Moore Park, Summerhill East",43.689574,-79.383160,10


There are 103 schools in neighbour, so we pick only with the highest ratings

In [617]:
Nyork_school_ratings_top = Nyork_school_ratings.loc[Nyork_school_ratings['Top School Rating']==10]

In [618]:
# Resetting the index

Nyork_school_ratings_top.reset_index(drop = True, inplace = True)
Nyork_school_ratings_top

,Neighbourhood,Latitude,Longitude,Top School Rating
0,"Moore Park, Summerhill East",43.689574,-79.383160,10
1,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,10
2,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,10
3,"The Danforth West, Riverdale",43.679557,-79.352188,10
4,Woodbine Heights,43.695344,-79.318389,10
5,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,10
6,Parkwoods,43.753259,-79.329656,10
7,"Dufferin, Dovercourt Village",43.669005,-79.442259,10


In [589]:
Nyork_school_ratings_top.shape

(8, 4)

There are 8 neighbours with the most high rating in North York

### Mapping Neighbour with 10 Rating Schools into Map

In [590]:
# create map
map_schools = folium.Map(location=[latitude_x, longitude_y], zoom_start=12)


markers_colors = []
for lat, lon, nei , ratings in zip(Nyork_school_ratings_top['Latitude'], 
                                   Nyork_school_ratings_top['Longitude'], 
                                   Nyork_school_ratings_top['Neighbourhood'], 
                                   Nyork_school_ratings_top['Top School Rating']):
    label = folium.Popup(str(nei) + ' Ratings ' + str(ratings), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_schools)
       
map_schools

,Neighbourhood,Latitude,Longitude,Top School Rating
0,"Moore Park, Summerhill East",43.689574,-79.383160,10
1,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,10
2,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,10
3,"The Danforth West, Riverdale",43.679557,-79.352188,10
4,Woodbine Heights,43.695344,-79.318389,10
5,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,10
6,Parkwoods,43.753259,-79.329656,10
7,"Dufferin, Dovercourt Village",43.669005,-79.442259,10


Conclusions: These 8 neighborhood in North York are locations with the most high rated schools